In [1]:
import pandas as pd
import numpy as np

import geopandas as gpd
from shapely.geometry import Point
from sklearn.metrics.pairwise import haversine_distances

## Create matrix of counties in the continental USA

In [2]:
"""
Continential US
Northernmost Point: 49.384472, -95.153389
Southernmost Point: 24.446667, -81.926667
Easternmost Point:  44.815389, -66.949778
Westernmost Point:  48.185,    -124.785

Southeast -80.433, 25.231
"""

northwest = np.array([49.4, -118.5])
southeast = np.array([24.4, -80.433]) #-80.433 is the SE tip of Florida

In [3]:
# Function to calculate distance between two coordinates
def get_distance(coord1, coord2):
    result = haversine_distances([np.radians(coord1), np.radians(coord2)])
    return result[0,1]*6371*0.621371 #convert to miles

c1 = [49.4, -124.9]
c2 = [48.4, -124.9]
row_scale = get_distance(c1, c2)
display(row_scale)

c1 = [24.4, -123.9]
c2 = [24.4, -124.9]
col_scale = get_distance(c1, c2)
display(col_scale)

# approx dimensions of array needed to create 1 mile * 1 mile grid
print('Approximate Continental USA Grid Dim:', abs(northwest - southeast)*np.array([row_scale, col_scale]))

69.0933027640562

62.92200560415788

Approximate Continental USA Grid Dim: [1727.3325691  2395.25198733]


In [4]:
latitudes = np.linspace(northwest[0], southeast[0], 1792)
longitudes = np.linspace(northwest[1], southeast[1], 2944)
long_med = np.median(longitudes)
base_med = long_med-longitudes[0]

display(latitudes)
display(longitudes)

coord_matrix = np.zeros([len(latitudes), len(longitudes), 2])
coord_matrix.shape

array([49.4       , 49.38604132, 49.37208264, ..., 24.42791736,
       24.41395868, 24.4       ])

array([-118.5       , -118.48706524, -118.47413048, ...,  -80.45886952,
        -80.44593476,  -80.433     ])

(1792, 2944, 2)

In [5]:
long_dist = np.array([get_distance((i, 0), (i, 1)) for i in latitudes])
display(long_dist)

num_long = len(longitudes)/long_dist
num_long

array([44.96381102, 44.97659043, 44.98936717, ..., 62.90809042,
       62.91504988, 62.9220056 ])

array([65.4748771 , 65.45627341, 65.43768417, ..., 46.79843213,
       46.79325544, 46.78808267])

In [6]:
def get_longs(long_dist):
    longs = np.linspace(0, long_dist, len(longitudes))
    longs += long_med - np.median(longs)*.9
    return longs

text_idx = len(latitudes)//2
test_lat = latitudes[text_idx]
test_longs = get_longs(num_long[text_idx])
display(test_lat, test_longs)

c1 = [test_lat, test_longs[0]]
c2 = [test_lat, test_longs[1]]
print('distance', get_distance(c1, c2))

36.89302065884981

array([-123.44145901, -123.42335582, -123.40525263, ...,  -70.19997871,
        -70.18187552,  -70.16377233])

distance 1.0003443636689682


In [7]:
for i in range(coord_matrix.shape[0]):
    coord_matrix[i,:,0] = latitudes[i]
    coord_matrix[i,:,1] = get_longs(num_long[i])

print(f'Coordinate Matrix {coord_matrix.shape}')
print(f'NW:\t{coord_matrix[0, 0]}')
print(f'NE:\t{coord_matrix[0, -1]}')
print(f'Center:\t{coord_matrix[coord_matrix.shape[0]//2, coord_matrix.shape[1]//2]}')
print(f'SW:\t{coord_matrix[-1, 0]}')
print(f'SE:\t{coord_matrix[-1, -1]}')

Coordinate Matrix (1792, 2944, 2)
NW:	[  49.4        -128.93019469]
NE:	[ 49.4       -63.4553176]
Center:	[ 36.89302066 -96.79356407]
SW:	[  24.4       -120.5211372]
SE:	[ 24.4        -73.73305453]


In [8]:
# download file from https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_500k.zip

shapefile = gpd.read_file("cb_2018_us_county_500k/cb_2018_us_county_500k.shp")
shapefile

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,"POLYGON ((-89.18137 37.04630, -89.17938 37.053..."
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,"POLYGON ((-84.44266 38.28324, -84.44114 38.283..."
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,"POLYGON ((-86.94486 37.07341, -86.94346 37.074..."
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,"POLYGON ((-84.12662 37.64540, -84.12483 37.646..."
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,"POLYGON ((-83.98428 38.44549, -83.98246 38.450..."
...,...,...,...,...,...,...,...,...,...,...
3228,31,073,00835858,0500000US31073,31073,Gosper,06,1186616237,11831826,"POLYGON ((-100.09510 40.43866, -100.08937 40.4..."
3229,39,075,01074050,0500000US39075,39075,Holmes,06,1094405866,3695230,"POLYGON ((-82.22066 40.66758, -82.19327 40.667..."
3230,48,171,01383871,0500000US48171,48171,Gillespie,06,2740719114,9012764,"POLYGON ((-99.30400 30.49983, -99.28234 30.499..."
3231,55,079,01581100,0500000US55079,55079,Milwaukee,06,625440563,2455383635,"POLYGON ((-88.06959 42.86726, -88.06959 42.872..."


In [9]:
def get_county(coord):
    pt = Point(coord[1], coord[0]) #flip the lat and long
    for idx, poly in enumerate(shapefile.geometry):
        if pt.within(poly):
            return int(shapefile.GEOID.iloc[idx])
    return 0

get_county([36.9, -95.87])

40147

In [10]:
def border_check(side):
    if side == 'left':
        idx = 0
    elif side == 'right':
        idx = -1
    else:
        raise Exception(f'Invalid Input: {side}') 
    
    for i in coord_matrix[:,idx]:
        if get_county(i) != 0:
            print(f'{side} border check failed. Found county at {i}, {get_county(i)}')
            return False
    print(f'{side} border check is Good')
    return True

In [11]:
border_check('left')

left border check is Good


True

In [12]:
border_check('right')

right border check is Good


True

In [13]:
# Single Thread

import time
start_time = time.time()

def fill_matrix():
    for row in range(coord_matrix.shape[0]):
        for col in range(coord_matrix.shape[0]):
            if col % 800 == 0:
                print(f'status: ({row}, {col})\ttime: {time.time()-start_time}')
            county_matrix[row, col] = get_county(coord_matrix[row, col])

#fill_matrix()

In [14]:
import itertools
import time

import multiprocessing
from multiprocessing import Pool, sharedctypes
cores = multiprocessing.cpu_count()

In [15]:
size = coord_matrix.shape[:2]
block_size = 64
print(size, block_size)

result = np.ctypeslib.as_ctypes(np.zeros(size, dtype=int))
shared_array = sharedctypes.RawArray(result._type_, result)

def fill_per_window(args):
    window_row, window_col = args
    tmp = np.ctypeslib.as_array(shared_array)

    print(f'time: {np.round(time.time()-start_time, 2)}\tstatus: ({window_row}, {window_col})')
    for row in range(window_row, window_row+block_size):
        for col in range(window_col, window_col+block_size):
            tmp[row, col] = get_county(coord_matrix[row, col])

window_idxs = [(i, j) for i, j in itertools.product(range(0, size[0], block_size),
                                                    range(0, size[1], block_size))]

len(window_idxs)

(1792, 2944) 64


1288

In [16]:
%%time
#This takes ~7 hours with 4 cores
start_time = time.time()

p = Pool(4)
res = p.map(fill_per_window, window_idxs)
result = np.ctypeslib.as_array(shared_array)
    
print(result.shape)

time: 0.11	status: (0, 0)
time: 0.12	status: (64, 2240)
time: 0.12	status: (192, 1536)
time: 0.12	status: (320, 832)
time: 34.6	status: (320, 896)
time: 76.89	status: (192, 1600)
time: 101.9	status: (320, 960)
time: 151.81	status: (64, 2304)
time: 152.74	status: (0, 64)
time: 180.55	status: (320, 1024)
time: 186.99	status: (192, 1664)
time: 268.12	status: (192, 1728)
time: 269.33	status: (320, 1088)
time: 311.8	status: (64, 2368)
time: 313.08	status: (0, 128)
time: 343.36	status: (192, 1792)
time: 390.85	status: (320, 1152)
time: 419.0	status: (192, 1856)
time: 443.4	status: (320, 1216)
time: 469.93	status: (64, 2432)
time: 471.25	status: (0, 192)
time: 482.16	status: (192, 1920)
time: 513.49	status: (320, 1280)
time: 560.55	status: (192, 1984)
time: 613.03	status: (320, 1344)
time: 620.52	status: (0, 256)
time: 620.7	status: (64, 2496)
time: 638.66	status: (192, 2048)
time: 700.79	status: (320, 1408)
time: 768.59	status: (0, 320)
time: 867.62	status: (192, 2112)
time: 868.39	status: (

time: 5754.98	status: (192, 192)
time: 5774.06	status: (64, 576)
time: 5785.42	status: (384, 2432)
time: 5801.98	status: (192, 256)
time: 5811.13	status: (64, 640)
time: 5830.58	status: (256, 2688)
time: 5841.39	status: (64, 704)
time: 5849.04	status: (192, 320)
time: 5863.62	status: (64, 768)
time: 5876.21	status: (192, 384)
time: 5877.83	status: (384, 2496)
time: 5894.59	status: (64, 832)
time: 5894.9	status: (256, 2752)
time: 5916.88	status: (384, 2560)
time: 5919.02	status: (192, 448)
time: 5926.16	status: (64, 896)
time: 5936.6	status: (64, 960)
time: 5947.92	status: (64, 1024)
time: 5951.75	status: (256, 2816)
time: 5962.08	status: (64, 1088)
time: 5972.97	status: (384, 2624)
time: 5992.02	status: (192, 512)
time: 6002.7	status: (64, 1152)
time: 6032.66	status: (384, 2688)
time: 6033.72	status: (256, 2880)
time: 6059.27	status: (64, 1216)
time: 6071.36	status: (192, 576)
time: 6114.2	status: (384, 2752)
time: 6126.35	status: (64, 1280)
time: 6148.02	status: (192, 640)
time: 6151.

time: 9899.8	status: (704, 1536)
time: 9932.84	status: (512, 128)
time: 9933.55	status: (576, 2304)
time: 9933.92	status: (832, 384)
time: 9971.36	status: (704, 1600)
time: 9973.74	status: (512, 192)
time: 9997.32	status: (576, 2368)
time: 10008.7	status: (832, 448)
time: 10026.78	status: (704, 1664)
time: 10047.75	status: (512, 256)
time: 10048.52	status: (576, 2432)
time: 10090.12	status: (576, 2496)
time: 10090.89	status: (704, 1728)
time: 10101.49	status: (832, 512)
time: 10113.04	status: (512, 320)
time: 10162.51	status: (576, 2560)
time: 10165.38	status: (704, 1792)
time: 10173.49	status: (512, 384)
time: 10193.9	status: (832, 576)
time: 10207.42	status: (576, 2624)
time: 10232.45	status: (832, 640)
time: 10242.39	status: (704, 1856)
time: 10246.49	status: (512, 448)
time: 10254.83	status: (576, 2688)
time: 10259.42	status: (832, 704)
time: 10303.15	status: (704, 1920)
time: 10307.27	status: (832, 768)
time: 10327.68	status: (512, 512)
time: 10334.99	status: (576, 2752)
time: 103

time: 13894.69	status: (1024, 576)
time: 13899.49	status: (1216, 2368)
time: 13910.46	status: (1152, 64)
time: 13912.4	status: (896, 2176)
time: 13960.07	status: (896, 2240)
time: 13960.14	status: (1216, 2432)
time: 13997.59	status: (1024, 640)
time: 14022.79	status: (896, 2304)
time: 14028.42	status: (1152, 128)
time: 14039.89	status: (1024, 704)
time: 14077.54	status: (1216, 2496)
time: 14088.91	status: (896, 2368)
time: 14095.92	status: (1024, 768)
time: 14146.55	status: (1152, 192)
time: 14154.8	status: (896, 2432)
time: 14164.73	status: (1024, 832)
time: 14195.61	status: (1216, 2560)
time: 14224.24	status: (896, 2496)
time: 14245.97	status: (1024, 896)
time: 14263.01	status: (1152, 256)
time: 14268.7	status: (896, 2560)
time: 14313.62	status: (1216, 2624)
time: 14322.51	status: (1024, 960)
time: 14330.2	status: (896, 2624)
time: 14344.42	status: (1152, 320)
time: 14375.82	status: (1024, 1024)
time: 14390.32	status: (1152, 384)
time: 14414.87	status: (1024, 1088)
time: 14432.32	sta

time: 18510.67	status: (1088, 1728)
time: 18525.1	status: (1216, 768)
time: 18539.62	status: (1280, 2752)
time: 18546.6	status: (1408, 2624)
time: 18578.58	status: (1344, 960)
time: 18617.01	status: (1216, 832)
time: 18658.01	status: (1280, 2816)
time: 18664.3	status: (1408, 2688)
time: 18679.48	status: (1344, 1024)
time: 18721.67	status: (1344, 1088)
time: 18722.72	status: (1216, 896)
time: 18753.04	status: (1344, 1152)
time: 18775.0	status: (1280, 2880)
time: 18780.88	status: (1408, 2752)
time: 18788.04	status: (1216, 960)
time: 18838.69	status: (1216, 1024)
time: 18840.4	status: (1344, 1216)
time: 18874.12	status: (1536, 1920)
time: 18893.01	status: (1344, 0)
time: 18898.08	status: (1408, 2816)
time: 18907.36	status: (1344, 1280)
time: 18992.22	status: (1536, 1984)
time: 18998.89	status: (1344, 1344)
time: 19010.81	status: (1344, 64)
time: 19015.22	status: (1408, 2880)
time: 19095.29	status: (1344, 1408)
time: 19112.17	status: (1536, 2048)
time: 19130.87	status: (1344, 128)
time: 19

time: 24701.45	status: (1600, 2240)
time: 24710.65	status: (1536, 256)
time: 24767.1	status: (1408, 2240)
time: 24789.35	status: (1728, 1088)
time: 24818.9	status: (1600, 2304)
time: 24828.46	status: (1536, 320)
time: 24877.39	status: (1408, 2304)
time: 24906.52	status: (1728, 1152)
time: 24935.77	status: (1600, 2368)
time: 24945.54	status: (1536, 384)
time: 24951.19	status: (1408, 2368)
time: 25002.82	status: (1408, 2432)
time: 25021.43	status: (1600, 2432)
time: 25023.85	status: (1728, 1216)
time: 25062.88	status: (1536, 448)
time: 25101.54	status: (1600, 2496)
time: 25109.31	status: (1408, 2496)
time: 25141.8	status: (1728, 1280)
time: 25183.91	status: (1536, 512)
time: 25203.97	status: (1600, 2560)
time: 25229.17	status: (1408, 2560)
time: 25261.86	status: (1728, 1344)
time: 25301.26	status: (1536, 576)
time: 25321.15	status: (1600, 2624)
time: 25373.11	status: (1728, 1408)
time: 25406.4	status: (1536, 640)
time: 25420.51	status: (1600, 2688)
time: 25466.31	status: (1728, 1472)
tim

In [17]:
np.savetxt('county_matrix.csv', result, delimiter=',', fmt="%d")